In [1]:
import boto3
import json
#import secrets

Note: If `boto3` gives you a credentials/permissions error, you might need to directly pass in the security keys when instantiating the clients. As a general programming best practice, you can create a `secrets.py` file with your security keys defined there, add it to your `.gitignore`, and avoid sharing it. 

Remember that these credentials change every 4 hours or when AWS Learner Lab is refreshed.

## Lambda Functions

In [2]:
# aws_lambda = boto3.client('lambda') ## Try this first, if it doesn't work then pass in your security keys

aws_lambda = boto3.client('lambda',
                          aws_access_key_id='asdfasdf',#secrets.AWS_ACCESS_KEY,
                          aws_secret_access_key='asdfsadf',#secrets.AWS_SECRET_KEY,
                          aws_session_token='asdfsadf',#secrets.AWS_SESSION_TOKEN
                        )
iam_client = boto3.client('iam',
                          aws_access_key_id='asdfasdf',#secrets.AWS_ACCESS_KEY,
                          aws_secret_access_key='asdfasdf',#secrets.AWS_SECRET_KEY,
                          aws_session_token='asdfasdf',#secrets.AWS_SESSION_TOKEN
                        )
role = iam_client.get_role(RoleName='LabRole')

In [3]:
# First we zip up the lambda function (and any dependencies required), and read it in here
with open('lambda_func.zip', 'rb') as f:
    lambda_zip = f.read()

In [4]:
# Create a new Lambda function in AWS, or update it based on the zip
try:
    response = aws_lambda.create_function(
        FunctionName='test2',
        Runtime='python3.11',
        Role=role['Role']['Arn'],
        Handler='lambda_func.lambda_handler',
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    response = aws_lambda.update_function_code(
        FunctionName='test2',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']
lambda_arn

'arn:aws:lambda:us-east-1:348752177325:function:test2'

In [5]:
event = json.dumps({'N':200}) # This is the input to the Lambda function

# We now trigger the Lambda function by manually invoking it
r = aws_lambda.invoke(FunctionName='test2',
                      InvocationType='RequestResponse',
                      Payload=event)
json.loads(r['Payload'].read())

{'statusCode': 200,
 'body': [2,
  3,
  5,
  7,
  11,
  13,
  17,
  19,
  23,
  29,
  31,
  37,
  41,
  43,
  47,
  53,
  59,
  61,
  67,
  71,
  73,
  79,
  83,
  89,
  97,
  101,
  103,
  107,
  109,
  113,
  127,
  131,
  137,
  139,
  149,
  151,
  157,
  163,
  167,
  173,
  179,
  181,
  191,
  193,
  197,
  199]}

## Step Functions

In [6]:
# This creates a Step Function which orchestrates more complicated workflows
## In this case, it runs multiple Lambda functions in parallel
sfn = boto3.client('stepfunctions',
                   aws_access_key_id='asdfasdf',#secrets.AWS_ACCESS_KEY,
                   aws_secret_access_key='asdfasdf',#secrets.AWS_SECRET_KEY,
                   aws_session_token='asdfasdf',#secrets.AWS_SESSION_TOKEN
                   )

def make_def(lambda_arn):
    definition = {
      "Comment": "My State Machine",
      "StartAt": "Map",
      "States": {
        "Map": {
          "Type": "Map",
          "End": True,
          "Iterator": {
            "StartAt": "Lambda Invoke",
            "States": {
              "Lambda Invoke": {
                "Type": "Task",
                "Resource": "arn:aws:states:::lambda:invoke",
                "OutputPath": "$.Payload",
                "Parameters": {
                  "Payload.$": "$",
                  "FunctionName": lambda_arn
                },
                "Retry": [
                  {
                    "ErrorEquals": [
                      "Lambda.ServiceException",
                      "Lambda.AWSLambdaException",
                      "Lambda.SdkClientException",
                      "Lambda.TooManyRequestsException",
                      "States.TaskFailed"
                    ],
                    "IntervalSeconds": 2,
                    "MaxAttempts": 6,
                    "BackoffRate": 2
                  }
                ],
                "End": True
              }
            }
          }
        }
      }
    }
    return definition

In [7]:
sf_def = make_def(lambda_arn)

# Create new State machine
try:
    response = sfn.create_state_machine(
        name='test_step_machine',
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn'],
        type='EXPRESS'
    )
except sfn.exceptions.StateMachineAlreadyExists:
    response = sfn.list_state_machines()
    state_machine_arn = [sm['stateMachineArn'] 
                         for sm in response['stateMachines'] 
                         if sm['name'] == 'test_step_machine'][0]
    response = sfn.update_state_machine(
        stateMachineArn=state_machine_arn,
        definition=json.dumps(sf_def),
        roleArn=role['Role']['Arn']
    )

In [8]:
response = sfn.list_state_machines()
response

{'stateMachines': [{'stateMachineArn': 'arn:aws:states:us-east-1:348752177325:stateMachine:test_step_machine',
   'name': 'test_step_machine',
   'type': 'EXPRESS',
   'creationDate': datetime.datetime(2024, 4, 12, 2, 23, 13, 473000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': '9f98d0b2-edff-48c4-8583-e25353ff32b4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9f98d0b2-edff-48c4-8583-e25353ff32b4',
   'date': 'Fri, 12 Apr 2024 07:23:18 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '186',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [9]:
state_machine_arn = [sm['stateMachineArn'] 
                     for sm in response['stateMachines'] 
                     if sm['name'] == 'test_step_machine'][0]

# Call Lambda functions 3 times in parallel
## Note: See Jon's post on Ed - DO NOT USE MORE THAN 10 LAMBDA FUNCTIONS IN PARALLEL
data = [{"N": i} for i in [10,29,34]]

sfn_response = sfn.start_sync_execution(
    stateMachineArn=state_machine_arn,
    name='sync_test',
    input=json.dumps(data)
)

print(sfn_response['output'])

[{"statusCode":200,"body":[2,3,5,7]},{"statusCode":200,"body":[2,3,5,7,11,13,17,19,23]},{"statusCode":200,"body":[2,3,5,7,11,13,17,19,23,29,31]}]
